In [ ]:
# Standard library
import os

# Third-party
import astropy.time as atime
from astropy import log as logger
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.gridspec as gridspec

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.util import quantity_from_hdf5
from thejoker.celestialmechanics import OrbitalParams, SimulatedRVOrbit, rv_from_elements
from thejoker.plot import plot_rv_curves, plot_corner, _truth_color

plt.style.use('../thejoker/thejoker.mplstyle')

In [ ]:
samples_filename = "../cache/experiment1-sample-jitter.h5"
data_filename = "../data/experiment1.h5"
plot_path = "../paper/figures/"
os.makedirs(plot_path, exist_ok=True)

In [ ]:
rv_unit = u.km/u.s

## read the data:

In [ ]:
with h5py.File(data_filename, 'r') as f:
    data = RVData.from_hdf5(f)
    truth_opars = OrbitalParams.unpack(f['truth_vector'])

(truth_opars.P, truth_opars.asini, truth_opars.ecc, truth_opars.omega.to(u.degree), 
 truth_opars.phi0.to(u.degree), truth_opars.v0.to(u.km/u.s))

In [ ]:
data.plot(rv_unit=u.km/u.s)
dmjd = data._t.max() - data._t.min()
t_grid = np.linspace(data._t.min() -0.25*dmjd, 
                     data._t.max() + 0.25*dmjd, 
                     1024)

truth_rv = truth_opars.rv_orbit(0).generate_rv_curve(t_grid + data.t_offset)
plt.plot(t_grid + data.t_offset, truth_rv.to(rv_unit).value, linestyle='--', 
         marker=None, linewidth=1, alpha=0.9, color=_truth_color)

## First read the orbital parameters sampled for this experiment:

In [ ]:
with h5py.File(samples_filename, 'r') as g:
    jitter = g.attrs['fixed_jitter']*u.m/u.s
    P_min = g.attrs['P_min']*u.day
    P_max = g.attrs['P_max']*u.day

    # read the orbital parameters
    opars = OrbitalParams.from_hdf5(g)
    samples = opars.pack(plot_transform=True)
#     samples[:,4] -= np.degrees((2*np.pi*data.t_offset / opars.P.to(u.day).value) % (2*np.pi))

In [ ]:
samples.shape

In [ ]:
plt.hist(samples[:,4]);

In [ ]:
plt.plot(samples[:,0], samples[:,4], ls='none')

## The Figure

In [ ]:
fig = plt.figure(figsize=(8,6.5))

gs = gridspec.GridSpec(2, 2)

# First, the RV curves
dmjd = data.t.mjd.max() - data.t.mjd.min()
t_grid = np.linspace(data.t.mjd.min() - 0.25*dmjd, 
                     data.t.mjd.max() + 0.25*dmjd, 
                     1024)

ax = fig.add_subplot(gs[0, :])
ax.set_title("Experiment 1")
plot_rv_curves(opars[:128], t_grid, rv_unit=rv_unit,
               ax=ax, plot_kwargs={'color': '#888888', 'zorder': -100})

data.plot(ax=ax, rv_unit=rv_unit, ecolor='k', markersize=3, 
          elinewidth=1, alpha=1., zorder=100)
ax.set_xlim(t_grid.min(), t_grid.max())
# ax.set_ylim(0, 30)
ax.set_ylim(truth_opars.v0.to(rv_unit).value - 15, truth_opars.v0.to(rv_unit).value + 15)

# Projections of the posterior samples:
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])

style = dict(alpha=0.25, marker='.', linestyle='none', zorder=-100, color='#888888')
ax2.plot(samples[:,5], samples[:,0], **style)
ax3.plot(samples[:,1], samples[:,0], **style)

# -------------------------------------------------------------
# now plot the truth:
truth_rv = truth_opars.rv_orbit(0).generate_rv_curve(t_grid)
truth_vec = truth_opars.pack(plot_transform=True)[0]
ax.plot(t_grid + data.t_offset, truth_rv.to(rv_unit).value, linestyle='--', 
        marker=None, linewidth=1, alpha=0.9, color=_truth_color)
ax2.scatter(truth_vec[5], truth_vec[0], marker='+', color=_truth_color, s=40, alpha=0.8)
ax3.scatter(truth_vec[1], truth_vec[0], marker='+', color=_truth_color, s=40, alpha=0.8)

lnP_lim = [2.5, 6.5]
lnasini_lim = [-0.5, 3.5]
ecc_lim = [-0.025, 1.025]

# ax2.set_xlim(lnP_lim)
ax2.set_ylim(lnP_lim)
ax2.set_xlabel(opars._latex_labels[5])
ax2.set_ylabel(opars._latex_labels[0])

# ax3.set_xlim(lnasini_lim)
# ax3.set_ylim(ecc_lim)
ax3.set_xlabel(opars._latex_labels[1])

# ax2.xaxis.set_ticks(range(3,6+1))
# ax3.xaxis.set_ticks(range(0,3+1))
# ax3.yaxis.set_ticklabels([])

for ax in fig.axes:
    ax.set_rasterization_zorder(-1)
    
fig.tight_layout()

fig.subplots_adjust(hspace=0.5)

# fig.savefig(os.path.join(plot_path, 'exp1-rv-curves.pdf'), dpi=128) # change to: dpi=256 for production?

## Full corner plot

In [ ]:
_med_v0 = np.median(samples[:,5])
_mad_v0 = np.median(np.abs(samples[:,5] - _med_v0))

ranges = [
    lnP_lim,
    lnasini_lim,
    ecc_lim,
    (0,360),
    (0,360),
    (_med_v0 - 5*_mad_v0, _med_v0 + 5*_mad_v0)
]

In [ ]:
fig = plot_corner(opars, alpha=0.5, truths=truth_vec, range=ranges, truth_color=_truth_color, 
                  plot_contours=False, plot_density=False, bins=32, color='#666666')
fig.suptitle("Experiment 1", fontsize=26)
fig.savefig(os.path.join(plot_path, 'exp1-corner.pdf'), dpi=128) # change to: dpi=256 for production?